In [21]:
import torch
import torchvision
from torchvision.datasets import MNIST
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [22]:
dataset=MNIST(root='data/',download=True)

In [23]:
len(dataset)

60000

In [24]:
test_dataset=MNIST(root='data/',train=False)
len(test_dataset)

10000

In [25]:
dataset[0]

(<PIL.Image.Image image mode=L size=28x28 at 0x24592DC6C70>, 5)

In [26]:
image,label=dataset[0]


In [27]:
import torchvision.transforms as transforms

In [28]:
dataset=MNIST(root='data/',train=True,transform=transforms.ToTensor())

In [29]:
img_tensor, label =dataset[0]
print(img_tensor.shape,label)

torch.Size([1, 28, 28]) 5


In [30]:
print(img_tensor[:,10:15,10:15])
print(torch.max(img_tensor),torch.min(img_tensor))

tensor([[[0.0039, 0.6039, 0.9922, 0.3529, 0.0000],
         [0.0000, 0.5451, 0.9922, 0.7451, 0.0078],
         [0.0000, 0.0431, 0.7451, 0.9922, 0.2745],
         [0.0000, 0.0000, 0.1373, 0.9451, 0.8824],
         [0.0000, 0.0000, 0.0000, 0.3176, 0.9412]]])
tensor(1.) tensor(0.)


In [31]:
n=6000
val_pct=0.2
n_val=int(val_pct*n)
idxs=np.random.permutation(n)
idxs


array([3384, 3877,  112, ..., 1696, 3153, 3446])

In [32]:


def split_indices(n, val_pct):
    n_val=int(val_pct*n)
    idxs=np.random.permutation(n)
    return idxs[n_val:], idxs[:n_val]

In [33]:
train_indices, val_indices=split_indices(len(dataset),val_pct=0.2)

In [34]:
print(len(train_indices),len(val_indices))
print('Sample val indices: ',val_indices[:20])

48000 12000
Sample val indices:  [57675 13189   950 12631 59950 56487 38307 34873 11000 26206 23293 38133
 53571 24910 20945 55979  2794 45216 55089 55472]


In [35]:
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data.dataloader import DataLoader

In [36]:
batch_size=100

train_sampler=SubsetRandomSampler(train_indices)
train_loader=DataLoader(dataset,batch_size,sampler=train_sampler)

val_sampler=SubsetRandomSampler(val_indices)
val_loader=DataLoader(dataset,batch_size,sampler=val_sampler)

In [37]:
import torch.nn as nn
input_size=28*28
num_classes=10

model=nn.Linear(input_size,num_classes)

In [38]:
print(model.weight.shape)
model.weight

torch.Size([10, 784])


Parameter containing:
tensor([[-0.0190,  0.0335, -0.0160,  ...,  0.0295, -0.0337, -0.0318],
        [ 0.0237,  0.0283, -0.0275,  ..., -0.0251, -0.0298,  0.0107],
        [-0.0151,  0.0279, -0.0010,  ...,  0.0236,  0.0262, -0.0155],
        ...,
        [ 0.0018,  0.0115,  0.0329,  ..., -0.0012,  0.0066, -0.0142],
        [-0.0054,  0.0028,  0.0064,  ...,  0.0188, -0.0314, -0.0292],
        [-0.0138, -0.0287, -0.0284,  ..., -0.0006,  0.0003,  0.0209]],
       requires_grad=True)

In [39]:
print(model.bias.shape)
model.bias

torch.Size([10])


Parameter containing:
tensor([ 0.0130, -0.0161,  0.0049,  0.0005,  0.0162,  0.0251,  0.0069, -0.0227,
         0.0175, -0.0055], requires_grad=True)

In [40]:
for images, labels in train_loader:
    print(labels)
    print(images.shape)
    outputs=model(images)
    break

tensor([3, 8, 2, 5, 0, 5, 3, 7, 0, 1, 5, 2, 9, 1, 1, 3, 7, 4, 6, 3, 4, 2, 5, 4,
        8, 9, 0, 2, 0, 8, 1, 9, 0, 7, 5, 3, 9, 9, 4, 4, 1, 5, 9, 8, 0, 2, 6, 4,
        3, 2, 4, 1, 3, 4, 8, 7, 4, 7, 2, 3, 6, 8, 3, 2, 5, 6, 2, 3, 6, 1, 0, 9,
        0, 1, 2, 2, 7, 3, 7, 8, 4, 5, 3, 3, 2, 5, 1, 4, 5, 2, 3, 0, 2, 3, 6, 7,
        3, 6, 1, 1])
torch.Size([100, 1, 28, 28])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (2800x28 and 784x10)

In [41]:
class MnistModel(nn.Module):

    def __init__(self):
        print(input_size)
        super().__init__()
        self.linear=nn.Linear(input_size,num_classes)

    def forward(self,xb):
        
        xb=xb.reshape(-1,784)
        out=self.linear(xb)
        return out
    
model=MnistModel()

784


In [42]:
print(model.linear.weight.shape, model.linear.bias.shape)
list(model.parameters())

torch.Size([10, 784]) torch.Size([10])


[Parameter containing:
 tensor([[ 0.0032,  0.0351, -0.0020,  ..., -0.0273, -0.0343,  0.0175],
         [ 0.0267,  0.0327, -0.0028,  ..., -0.0047,  0.0209,  0.0318],
         [ 0.0321, -0.0186, -0.0260,  ...,  0.0270, -0.0352, -0.0332],
         ...,
         [-0.0233, -0.0131, -0.0071,  ..., -0.0009,  0.0074,  0.0125],
         [ 0.0008, -0.0056,  0.0250,  ..., -0.0077,  0.0085,  0.0291],
         [ 0.0303, -0.0031,  0.0347,  ...,  0.0114,  0.0009,  0.0321]],
        requires_grad=True),
 Parameter containing:
 tensor([ 0.0295, -0.0116, -0.0014,  0.0228,  0.0355, -0.0139, -0.0062,  0.0016,
          0.0091,  0.0356], requires_grad=True)]

In [43]:
for images, labels in train_loader:
    outputs=model(images)
    
    break
print('outputs.shape: ',outputs.shape)
print("Sample outputs:\n",outputs[:2].data)


outputs.shape:  torch.Size([100, 10])
Sample outputs:
 tensor([[ 0.2248,  0.0419,  0.1310,  0.0837,  0.1260,  0.2008,  0.1840,  0.0743,
         -0.1357, -0.0686],
        [ 0.2260,  0.1263,  0.1603, -0.1607,  0.2225,  0.2124, -0.0124,  0.0868,
         -0.1054, -0.1007]])


In [44]:
import torch.nn.functional as F

In [45]:
probs=F.softmax(outputs, dim=1)
print("Sample probabilities:\n",probs[:2].data)
print("Sum: ",torch.sum(probs[0]).item())

Sample probabilities:
 tensor([[0.1142, 0.0951, 0.1040, 0.0992, 0.1035, 0.1115, 0.1096, 0.0982, 0.0796,
         0.0852],
        [0.1163, 0.1052, 0.1089, 0.0790, 0.1159, 0.1147, 0.0916, 0.1012, 0.0835,
         0.0839]])
Sum:  1.0
